In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
import mlflow
import pickle
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [5]:
# load the dataset

data= pd.read_csv('merged.csv',parse_dates =True)

# only the date and sales columns will be used for LSTM
data= data.groupby("Date").agg({'Sales':'mean'})

data

,Sales
Date,
2013-01-01,76.857271
2013-01-02,6050.476233
2013-01-03,5577.322870
2013-01-04,5832.756054
2013-01-05,5178.539910
...,...
2015-07-27,8694.100448
2015-07-28,7742.983857
2015-07-29,7326.593722


In [6]:
# Normalize the sales data using Standardscaler
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array

In [7]:
SIZE = len(data.DataScaled)  # Get the total number of data points in the 'DataScaled' column

# Define parameters for time series analysis
WINDOW_SIZE = 48  # Set the size of the time window, indicating the number of past data points to consider

BATCH_SIZE = SIZE - WINDOW_SIZE * 2  # Set the batch size for training, leaving a gap equal to twice the window size for validation

EPOCHS = 200  # Set the number of training epochs

In [8]:
# Extract the indices for the training set
DateTrain = data.index.values[0:BATCH_SIZE]

# Extract the indices for the validation set
DateValid = data.index.values[BATCH_SIZE:]

# Extract the training data and convert to float32
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')

# Extract the validation data and convert to float32
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Reshape date series to have a single column for compatibility with the model
DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

# Print the shapes of the training and validation sets for date series
print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()

# Print the shapes of the training and validation sets for the sales series
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series: ", XValid.shape)


Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series:  (96,)


In [9]:
# Set the seed for reproducibility using TensorFlow's random number generator
tf.random.set_seed(1234)

# Add an extra dimension to the training data (XTrain) using tf.expand_dims
# This is typically done to make the data compatible with the input requirements of certain models, such as LSTM
series = tf.expand_dims(XTrain, axis=-1)

# Print the shape of the resulting data with the added dimension
print("Shape after adding an extra dimension: ", series.shape)

Shape after adding an extra dimension:  (846, 1)


In [10]:
#  create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [11]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)

In [12]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [13]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))

In [14]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

In [15]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [16]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset


In [17]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

# Model Training


In [18]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 8)           320       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 533 (2.08 KB)
Trainable params: 533 (2.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

2024/02/05 19:19:37 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/02/05 19:19:37 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'edf5fa2a615b499bbf3f53fe1f08579c', which will track hyperparameters, performance metrics, model artifacts,

Epoch 1/200

1/1 [==============================] - 10s 10s/step - loss: 0.4132 - val_loss: 0.4028
Epoch 2/200
1/1 [==============================] - 0s 344ms/step - loss: 0.4121 - val_loss: 0.4015
Epoch 3/200
1/1 [==============================] - 0s 302ms/step - loss: 0.4111 - val_loss: 0.4003
Epoch 4/200
1/1 [==============================] - 0s 312ms/step - loss: 0.4102 - val_loss: 0.3989
Epoch 5/200
1/1 [==============================] - 0s 314ms/step - loss: 0.4092 - val_loss: 0.3976
Epoch 6/200
1/1 [==============================] - 0s 320ms/step - loss: 0.4083 - val_loss: 0.3963
Epoch 7/200
1/1 [==============================] - 0s 314ms/step - loss: 0.4073 - val_loss: 0.3950
Epoch 8/200
1/1 [==============================] - 0s 315ms/step - loss: 0.4063 - val_loss: 0.3937
Epoch 9/200
1/1 [==============================] - 0s 323ms/step - loss: 0.4054 - val_loss: 0.3923
Epoch 10/200
1/1 [==============================] - 0s 316ms/step - loss: 0.4045 - val_loss: 0.3910
Epoch 11/

INFO:tensorflow:Assets written to: C:\Users\VARUN\AppData\Local\Temp\tmpa4lf5sf1\model\data\model\assets
2024/02/05 19:21:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\ProgramData\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [27]:
from time import gmtime, strftime

In [28]:
time = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model.save(f'../models/LSTM_sales-{time}.pkl')


INFO:tensorflow:Assets written to: ../models/LSTM_sales-2024-02-05-13-57-34.pkl\assets


INFO:tensorflow:Assets written to: ../models/LSTM_sales-2024-02-05-13-57-34.pkl\assets
